In [11]:
try:
    from sdRDM import DataModel
    from sdRDM.database import build_sql_database
    
except ModuleNotFoundError:
    # Install package if not given
    import subprocess
    import sys
    
    subprocess.call([sys.executable, "-m", "pip", "install", "git+https://github.com/JR-1991/software-driven-rdm.git"])

### Fetching the data model

The PyEED data model can be inspected [here](https://github.com/PyEED/pyeed-data-model) which incorporates a [markdown file](https://github.com/PyEED/pyeed-data-model/blob/main/specifications/model.md) that defines the model. By using sdRDM's ```DataModel``` class and ```from_git``` method we can generate the corresponding Python code in-memory and use the model right away. See the printed tree to verify that the code is present.

In [12]:
# pyeed=DataModel.from_markdown('./specifications/model.md')
pyeed = DataModel.from_git("https://github.com/JR-1991/pyeed-data-model.git", tag="patch-1")

pyeed.ProteinSequence.visualize_tree()

ProteinSequence
├── id
├── name
├── amino_acid_sequence
├── ncbi_id
├── uniprot_id
├── pdb_id
├── organism
│   └── Organism
│       ├── id
│       ├── name
│       └── ncbi_taxonomy_id
├── substrate
├── domain
│   └── Domain
│       ├── id
│       ├── name
│       ├── start_position
│       └── end_position
├── reference_sequence
├── equivalence
│   └── Equivalence
│       ├── id
│       ├── reference_position
│       └── sequence_position
└── annotation
    └── Annotation
        ├── id
        ├── start_position
        ├── function
        └── end_position


### Building the SQL database

Next, we are going to use the ```build_sql_database``` function to set up an SQLite database file. This one will be used later on to populate data from our model. The database will construct a table for each object/attribute and thus facilitate an easy transfer from an application to the database.

In [13]:
build_sql_database(pyeed.ProteinSequence, pyeed.DNASequence, loc="./test.db")

### Importing PAZy database

Next we are importing the PAZy database from an excel file with all the important information


In [14]:
import subprocess
subprocess.call([sys.executable, "-m", "pip", "install", "openpyxl"])
subprocess.call([sys.executable, "-m", "conda", "install", "pandas"])
import pandas as pd
PAZy = pd.read_excel('./PAZy_DB.xlsx')

/Users/jara1991/opt/anaconda3/envs/softdata/bin/python: No module named conda


### Creating a dataset and populating the database

In order to demonstrate how to populate the database using our model, we are going to construct a small dataset using the data model we just loaded. 

Finally, we are going to add the data to our previously created database by using the datasets ```to_sql``` method where we also submit the location of our database file. This will create an Object Relation Model (ORM) that represents the Database structure and map the values present in our dataset to the corresponding tables.

In [ ]:
for index, row in PAZy.iterrows():
    dataset = pyeed.ProteinSequence(
        name=row["Enzyme"],
        pdb_id=[row["PDB_Accession"]],
        amino_acid_sequence=row["Sequence"],
        substrate=[row["Substrate"]],
        ncbi_id=row["NCBI_Accession"],
        organism=pyeed.Organism(
            name=row["Microbial host"],
            ncbi_taxonomy_id=row["NCBI_Accession"]
        )
    )
    
    dataset.to_sql(loc="./test.db")

In [7]:
pyeed.Organism.visualize_tree()

Organism
├── id
├── name
└── ncbi_taxonomy_id
